In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.append('C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import matplotlib.pyplot as plt
from hernando_behavioural_analysis_dev.BehaviouralAnalysis.utils import custom_functions as cuf
import matplotlib.cm as cm
import numpy as np
import os
import matplotlib
import pandas as pd
import pickle
from utils.post_processing_utils import *
from utils.plotting import calculate_error_bars
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
def moving_average(x, window=50):
    rolling_average = np.empty(len(x))
    rolling_average[:] = np.nan
    for i in range(window, len(x)):
        win = range((i - window), i)
        rolling_average[i] = np.mean(x[win])
    return rolling_average

In [34]:
processed_data_dir = os.path.join('T:\\photometry_2AC\\processed_data\\state_change_data')
state_change_data_file = os.path.join(processed_data_dir, 'value_switch_nacc_mice.csv')
pre_pc = []
post_pc = []
mice = [ 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo34', 'SNL_photo35']
mouse_ids = []
session_ids = []
moving_avs = []
reward_blocks = []
performance_to_delvalued_tone = []
#mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        traces, trial_data = open_experiment(experiment_to_process)
        trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1:
            side = -1
            devalued_trial_type = 7
        elif post_reward_side == 5:
            side = 1
            devalued_trial_type = 1
        devalued_trials = post_trials[post_trials['Trial type'] == devalued_trial_type]
        performance = np.mean(devalued_trials['Trial outcome'].values) * 100
        pre_bias = cuf.RBias(pre_trials['First response'], pre_trials['First choice correct'])* side
        post_bias = cuf.RBias(post_trials['First response'], post_trials['First choice correct']) *side
        post_pc.append(post_bias)
        pre_pc.append(pre_bias)
        session_ids.append(date)
        mouse_ids.append(mouse_id)
        reward_blocks.append(post_reward_side)
        performance_to_delvalued_tone.append(performance)
        


In [35]:
mice=['SNL_photo70', 'SNL_photo72', 'SNL_photo37', 'SNL_photo44', 'SNL_photo43']
state_change_data_file = os.path.join(processed_data_dir,'value_switch_all_tail_mice_test.csv')
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        traces, trial_data = open_experiment(experiment_to_process)
        trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1:
            side = -1
            devalued_trial_type = 7
        elif post_reward_side == 5:
            side = 1
            devalued_trial_type = 1
        devalued_trials = post_trials[post_trials['Trial type'] == devalued_trial_type]
        performance = np.mean(devalued_trials['Trial outcome'].values) * 100
        pre_bias = cuf.RBias(pre_trials['First response'], pre_trials['First choice correct'])* side
        post_bias = cuf.RBias(post_trials['First response'], post_trials['First choice correct']) *side
        post_pc.append(post_bias)
        pre_pc.append(pre_bias)
        session_ids.append(date)
        mouse_ids.append(mouse_id)
        reward_blocks.append(post_reward_side)
        performance_to_delvalued_tone.append(performance)


In [36]:
behavioural_change_performance = {}
behavioural_change_performance['mouse'] = mouse_ids
behavioural_change_performance['session'] = session_ids
behavioural_change_performance['performance to devalued tone'] = performance_to_delvalued_tone 
behavioural_change_performance_df = pd.DataFrame(behavioural_change_performance)

per_mouse_performance = behavioural_change_performance_df.groupby(['mouse'])[['performance to devalued tone']].apply(np.mean)
per_mouse_performance = per_mouse_performance.reset_index()

C:\Users\francescag\AppData\Local\Continuum\miniconda3\envs\py38\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [37]:
np.mean(per_mouse_performance['performance to devalued tone'])

81.9938010150495

In [38]:
np.std(per_mouse_performance['performance to devalued tone'])

6.239987618045049

In [39]:
post_trials

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
944,100.0,7.0,1.0,1.0,1.0,TrialStart,936.2984,936.3084,1.0,1.0,0.0,936.2984,938.9149,0.0,1.0,5.0,0.0
951,101.0,1.0,1.0,1.0,1.0,TrialStart,939.1054,939.1154,1.0,1.0,0.0,939.1054,940.6004,1.0,1.0,5.0,1.0
959,102.0,7.0,1.0,1.0,1.0,TrialStart,940.7612,940.7712,2.0,1.0,1.0,940.7612,950.2004,1.0,2.0,5.0,1.0
972,103.0,1.0,1.0,1.0,1.0,TrialStart,950.3738,950.3838,1.0,2.0,1.0,950.3738,957.8357,1.0,1.0,5.0,1.0
985,104.0,7.0,1.0,1.0,1.0,TrialStart,957.9962,958.0062,2.0,1.0,1.0,957.9962,962.0884,1.0,2.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855,307.0,1.0,1.0,1.0,1.0,TrialStart,2164.2924,2164.3024,2.0,2.0,1.0,2164.2924,2229.3704,0.0,2.0,5.0,0.0
2872,308.0,1.0,1.0,1.0,1.0,TrialStart,2229.5529,2229.5629,1.0,2.0,0.0,2229.5529,2243.7099,1.0,1.0,5.0,1.0
2880,309.0,1.0,1.0,1.0,1.0,TrialStart,2243.8720,2243.8820,1.0,1.0,1.0,2243.8720,2247.5658,1.0,1.0,5.0,1.0
2887,310.0,1.0,1.0,1.0,1.0,TrialStart,2247.7889,2247.7989,NaN,1.0,1.0,2247.7889,2291.2642,0.0,NaN,5.0,0.0


In [10]:
post_reward_side

5.0

In [103]:
timepoints = all_reward_block_data['time points'].iloc[0]
font = {'size'   : 12}

matplotlib.rc('font', **font)

mouse_name = 'SNL_photo32'

fig, ax = plt.subplots(2,1) #, figsize=(10,16))
plot_mean_trace_for_condition(ax[0], all_reward_block_data[all_reward_block_data['mouse'] == mouse_name], timepoints,
                              'contra reward amount', error_bar_method='ci', save_location=processed_data_dir)
lg1 = ax[0].legend(title='Absolute value (ul)',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')
plot_mean_trace_for_condition(ax[1], all_reward_block_data[all_reward_block_data['mouse'] == mouse_name], timepoints,
                              'relative reward amount', error_bar_method = 'ci', save_location=processed_data_dir)
lg2 = ax[1].legend(title='Relative value (ul)',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')

plt.tight_layout()


NameError: name 'all_reward_block_data' is not defined

In [40]:
behavioural_change = {}
behavioural_change['mouse'] = mouse_ids
behavioural_change['session'] = session_ids
behavioural_change['pre bias'] = pre_pc 
behavioural_change['post bias'] = post_pc
behavioural_change_df = pd.DataFrame(behavioural_change)

In [41]:
df_for_plot = behavioural_change_df.groupby(['mouse'])[['pre bias', 'post bias']].apply(np.mean)*100
df_for_plot = df_for_plot.reset_index()

C:\Users\francescag\AppData\Local\Continuum\miniconda3\envs\py38\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [42]:
df_for_plot1 = df_for_plot.set_index('mouse').transpose()


In [43]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, color='gray', legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color, alpha=0.7)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [44]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[2,2])
francescas_plot(ax, df_for_plot1, mean_line_color='#7FB5B5', mean_linewidth=6, show_err_bar=False)
plt.xticks([0, 1], ['equal\nrewards', 'unequal\nrewards'])
plt.ylabel('Bias towards BIG side (%)')

y = df_for_plot1.to_numpy().max() + .2
h = 1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
ax.text(.5, y+h, '**', ha='center', fontsize=8)
plt.tight_layout()
figure_dir = r'T:\paper\revisions\value switch behaviour'
plt.tight_layout()
#plt.savefig(os.path.join(figure_dir, 'bias before and after with added mice.pdf'))

plt.show()

In [45]:
import scipy.stats as stats
pre_data = df_for_plot1.T['pre bias'].values
post_data = df_for_plot1.T['post bias'].values
stat, pval = stats.ttest_rel(pre_data, post_data)
#pval *= .5 # multiply by .5 bc doing a one-sided t-test! we have a hypothesis that it's going a certain way 
pval

0.0017325467666258832

In [46]:
df_for_plot1

mouse,SNL_photo30,SNL_photo31,SNL_photo32,SNL_photo34,SNL_photo35,SNL_photo37,SNL_photo43,SNL_photo44,SNL_photo70,SNL_photo72
pre bias,-0.666667,-2.000000,-3.333333,-0.166667,-0.428571,2.250000,-6.000000,-4.750000,5.800000,-1.600000
post bias,3.676233,11.497722,3.787264,3.342535,4.634689,5.836747,5.127002,8.636544,3.924516,6.417621


In [18]:
np.mean(df_for_plot1.T['post bias'].values)

5.688087278321691

In [17]:
np.mean(100 - (df_for_plot1.T['post bias'].values + 50))

44.31191272167831

In [110]:
differences = pre_data - post_data
stats.shapiro(differences)


ShapiroResult(statistic=0.9447067379951477, pvalue=0.6064687371253967)

In [56]:
df_for_plot1.to_numpy().max() + .2

11.697722384256606

In [5]:
def calc_per_mouse_mean(mouse_moving_avs):
    num_sessions = len(mouse_moving_avs)
    num_trials = [len(m) for m in mouse_moving_avs]
    min_num_trials = min(num_trials)
    all_sessions = np.empty((num_sessions, min_num_trials))
    all_sessions[:] = np.nan
    for i, session_data in enumerate(mouse_moving_avs):
        all_sessions[i, :min_num_trials] = session_data[:min_num_trials]
    moving_av = np.nanmean(all_sessions, axis=0)
    return moving_av

In [6]:
def align_mulitple_mice_moving_avs(mice, moving_avs):
    num_mice = len(mice)
    num_trials = [len(m) for m in moving_avs]
    min_num_trials = min([len(m) for m in moving_avs])
    all_mice = np.empty((num_mice, min_num_trials))
    all_mice[:] = np.nan
    for i, mouse_data in enumerate(moving_avs):
        all_mice[i, :min_num_trials] = mouse_data[:min_num_trials]
    return all_mice, min_num_trials

In [15]:

moving_avs = []
response_times = []
missed_trials = []
performance = []

mice=['SNL_photo70', 'SNL_photo72', 'SNL_photo37', 'SNL_photo44', 'SNL_photo43','SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    mouse_moving_avs = []
    mouse_response_times = []
    mouse_missed_trials = []
    mouse_performance = []
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        traces, trial_data = open_experiment(experiment_to_process)
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        red_trial_data_for_missed_trials = trial_data[trial_data['State name'] == 'TrialStart']
        red_trial_data_for_missed_trials.loc[trial_data['Trial outcome'] == 1, 'Trial outcome'] = 0
        red_trial_data_for_missed_trials.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 1
        red_trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        response_trial_data = trial_data[trial_data['State name'] == 'WaitForResponse']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1: #left
            side = -1
            #trial_data.loc[trial_data['First response'] == 2, 'Trial outcome'] = 0
        elif post_reward_side == 5: #right
            side = 1
            #trial_data.loc[trial_data['First response'] == 1, 'Trial outcome'] = 0
        mouse_moving_avs.append(cuf.CalculateRBiasWindow(red_trial_data['First response'].reset_index(drop=True), red_trial_data['First choice correct'].reset_index(drop=True), 20) *side*100)
        mouse_response_times.append(moving_average(response_trial_data['Time end'].values - response_trial_data['Time start'].values, window=20))
        mouse_missed_trials.append(moving_average(red_trial_data_for_missed_trials['Trial outcome'].values, window=20) * 100, )
        mouse_performance.append(moving_average(red_trial_data['Trial outcome'].values, window=20) * 100)
    response_times.append(calc_per_mouse_mean(mouse_response_times))
    moving_avs.append(calc_per_mouse_mean(mouse_moving_avs))
    performance.append(calc_per_mouse_mean(mouse_performance))
    missed_trials.append(calc_per_mouse_mean(mouse_missed_trials))
    
        




C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarning: Mean of empty slice
  moving_av = np.nanmean(all_sessions, axis=0)
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\421496107.py:9: RuntimeWarni

In [218]:
post_trials['Trial outcome'].mean()

0.9366197183098591

In [220]:
pre_trials['Trial outcome'].mean()

0.98

In [12]:
def plot_rolling_mean(var, mice, moving_avs, y_axis_label='% bias to big reward side'):
    
    all_mice, min_num_trials = align_mulitple_mice_moving_avs(mice, moving_avs)
    error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
                                                                     all_mice,
                                                                    error_bar_method='sem')
    font = {'size': 8}
    matplotlib.rc('font', **font)
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['font.sans-serif'] = 'Arial'
    matplotlib.rcParams['font.family']


    fig, axs = plt.subplots(1, 1, figsize=[2.5, 2])
    [axs.plot(m[:min_num_trials], alpha=0.5, c='gray', lw=0.5) for m in moving_avs]
    axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)
    axs.fill_between(np.arange(0, min_num_trials), error_bar_lower, error_bar_upper, alpha=0.5,
                                facecolor='#7FB5B5', linewidth=0)

    axs.axvline(100, color='k')
    axs.set_xlabel('trial number')
    axs.set_ylabel(y_axis_label)
    axs.spines['right'].set_visible(False)
    axs.spines['top'].set_visible(False)
    plt.tight_layout()
    figure_dir = r'T:\paper\revisions\value switch behaviour'
    plt.tight_layout()
    plt.savefig(os.path.join(figure_dir, '{} developing over trials.pdf'.format(var)))

    plt.show()

# Rolling mean bias

In [23]:
plot_rolling_mean('bias', mice, moving_avs, y_axis_label='% bias to big reward side')


C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


# Rolling mean response time

In [18]:
plot_rolling_mean('response time', mice, response_times, y_axis_label='response time (s)')


C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


In [22]:
plot_rolling_mean('performance', mice, performance, y_axis_label='% correct')


C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


In [21]:
plot_rolling_mean('missed trials', mice, missed_trials, y_axis_label='% missed trials')


C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_5416\2414900081.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)
